In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [6]:
# Get Relevant Fields
all_df = df[["loan_amnt", "int_rate", "installment", "home_ownership", "annual_inc", "loan_status","dti", "total_pymnt", "tot_cur_bal", "avg_cur_bal"]]
all_df

,loan_amnt,int_rate,installment,home_ownership,annual_inc,loan_status,dti,total_pymnt,tot_cur_bal,avg_cur_bal
0,10500.0,0.1719,375.35,RENT,66000.0,low_risk,27.24,355.29,38199.0,4775.0
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,low_risk,20.23,873.53,228119.0,13419.0
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,low_risk,24.26,485.44,45684.0,5711.0
3,10000.0,0.1640,353.55,RENT,92000.0,low_risk,31.44,335.33,68784.0,6878.0
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,low_risk,18.76,484.36,177745.0,13673.0
...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,RENT,26000.0,low_risk,9.60,1019.42,6798.0,755.0
68813,12000.0,0.2727,368.37,RENT,63000.0,low_risk,29.07,1086.93,60350.0,7544.0
68814,5000.0,0.1992,185.62,MORTGAGE,52000.0,low_risk,14.86,571.33,18611.0,3722.0
68815,40000.0,0.0646,1225.24,MORTGAGE,520000.0,low_risk,9.96,3661.36,864480.0,41166.0


In [7]:
# Convert text field to number
all_df = pd.get_dummies(all_df, columns = ['home_ownership'])
all_df

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,total_pymnt,tot_cur_bal,avg_cur_bal,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,355.29,38199.0,4775.0,0,0,0,1
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,873.53,228119.0,13419.0,0,1,0,0
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,485.44,45684.0,5711.0,0,1,0,0
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,335.33,68784.0,6878.0,0,0,0,1
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,484.36,177745.0,13673.0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,26000.0,low_risk,9.60,1019.42,6798.0,755.0,0,0,0,1
68813,12000.0,0.2727,368.37,63000.0,low_risk,29.07,1086.93,60350.0,7544.0,0,0,0,1
68814,5000.0,0.1992,185.62,52000.0,low_risk,14.86,571.33,18611.0,3722.0,0,1,0,0
68815,40000.0,0.0646,1225.24,520000.0,low_risk,9.96,3661.36,864480.0,41166.0,0,1,0,0


In [8]:
# Convert Target text field to number
from sklearn.preprocessing import LabelEncoder
all_df['loan_status_code'] = LabelEncoder().fit_transform(all_df.loan_status)
all_df.drop(columns=['loan_status'], inplace = True)
all_df.rename(columns = {'loan_status_code': 'loan_status'}, inplace = True)
all_df

,loan_amnt,int_rate,installment,annual_inc,dti,total_pymnt,tot_cur_bal,avg_cur_bal,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,loan_status
0,10500.0,0.1719,375.35,66000.0,27.24,355.29,38199.0,4775.0,0,0,0,1,1
1,25000.0,0.2000,929.09,105000.0,20.23,873.53,228119.0,13419.0,0,1,0,0,1
2,20000.0,0.2000,529.88,56000.0,24.26,485.44,45684.0,5711.0,0,1,0,0,1
3,10000.0,0.1640,353.55,92000.0,31.44,335.33,68784.0,6878.0,0,0,0,1,1
4,22000.0,0.1474,520.39,52000.0,18.76,484.36,177745.0,13673.0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,26000.0,9.60,1019.42,6798.0,755.0,0,0,0,1,1
68813,12000.0,0.2727,368.37,63000.0,29.07,1086.93,60350.0,7544.0,0,0,0,1,1
68814,5000.0,0.1992,185.62,52000.0,14.86,571.33,18611.0,3722.0,0,1,0,0,1
68815,40000.0,0.0646,1225.24,520000.0,9.96,3661.36,864480.0,41166.0,0,1,0,0,1


# Split the Data into Training and Testing

In [9]:
# Create our features
X = all_df.drop(columns='loan_status')

# Create our target
y = all_df['loan_status']

In [10]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,total_pymnt,tot_cur_bal,avg_cur_bal,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,970.639473,1.634976e+05,14142.936193,0.009285,0.526309,0.106747,0.357659
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,1229.563301,1.723699e+05,15863.878993,0.095914,0.499311,0.308793,0.479314
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,2.350000e+02,20.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,403.760000,3.570000e+04,3505.000000,0.000000,0.000000,0.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,710.790000,9.960600e+04,8261.000000,0.000000,1.000000,0.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,1214.600000,2.472060e+05,19859.000000,0.000000,1.000000,0.000000,1.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,41418.981165,3.140082e+06,448583.000000,1.000000,1.000000,1.000000,1.000000


In [12]:
# Check the balance of our target values
y.value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [14]:
# Scaling X data
# Creating the scaler instance
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()
# Fitting the scaler
X_scaled = data_scaler.fit_transform(X)
X_scaled

array([[-0.60109272,  0.91798436, -0.36555835, ..., -1.054078  ,
        -0.34569279,  1.34013488],
       [ 0.80978726,  1.50182932,  1.55674724, ...,  0.94869639,
        -0.34569279, -0.74619355],
       [ 0.32327692,  1.50182932,  0.17089176, ...,  0.94869639,
        -0.34569279, -0.74619355],
       ...,
       [-1.13625409,  1.4852074 , -1.02420508, ...,  0.94869639,
        -0.34569279, -0.74619355],
       [ 2.26931827, -1.31143071,  2.58483046, ...,  0.94869639,
        -0.34569279, -0.74619355],
       [-0.06593135, -0.30372679, -0.45231101, ...,  0.94869639,
        -0.34569279, -0.74619355]])

In [15]:
# Split data into training and testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, 
                                                    y, 
                                                    random_state=1,
                                                    stratify=y)
X_train.shape

(51612, 12)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [17]:
# Create a random forest classifier.
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=1)

In [18]:
# Fitting the model
rf_model = rf_model.fit(X_train, y_train)

In [20]:
# Making predictions using the testing data.
y_pred = rf_model.predict(X_test)

In [21]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.6663161584297231

In [22]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,29,58
Actual 1,12,17106


In [23]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.71      0.33      1.00      0.45      0.58      0.31        87
          1       1.00      1.00      0.33      1.00      0.58      0.36     17118

avg / total       1.00      1.00      0.34      1.00      0.58      0.36     17205



In [24]:
# List the features sorted in descending order by feature importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.3336491750500604, 'total_pymnt'),
 (0.11941149627150663, 'tot_cur_bal'),
 (0.11812335637928695, 'dti'),
 (0.11635058247974424, 'avg_cur_bal'),
 (0.08973464085624046, 'installment'),
 (0.0852901775693437, 'annual_inc'),
 (0.05992721799128182, 'loan_amnt'),
 (0.05145866493427285, 'int_rate'),
 (0.009111533862956806, 'home_ownership_RENT'),
 (0.008310922596464746, 'home_ownership_MORTGAGE'),
 (0.006834020855813158, 'home_ownership_OWN'),
 (0.0017982111530282995, 'home_ownership_ANY')]

### Easy Ensemble AdaBoost Classifier

In [25]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(random_state=1)
eec.fit(X_train, y_train)

EasyEnsembleClassifier(random_state=1)

In [26]:
# Make Predictions
y_pred = eec.predict(X_test)

In [27]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.6837750945767915

In [28]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,29,58
Actual 1,12,17106


In [29]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.63      0.74      0.02      0.68      0.46        87
          1       1.00      0.74      0.63      0.85      0.68      0.47     17118

avg / total       0.99      0.73      0.63      0.84      0.68      0.47     17205

